In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import fcluster
from matplotlib import transforms
from matplotlib.patches import Rectangle
import matplotlib.colors as clr

import tensorflow as tf
import keras
from keras.layers import *
from keras.regularizers import *
import keras.backend as K
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from keras.callbacks import EarlyStopping
from keras import metrics
import keras.backend as K

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# session = tf.Session(config=config)
# K.set_session(session)

### Load model

In [ ]:
model_load_path = ''
model_name = ''


print("Model loaded ===", str(model_name))

### Analysis 1. latent features - significant genes

In [ ]:
# LINCS gene info
gene_annotation = pd.read_csv('./data/lincs_gene_list.csv', index_col=0)
# TWOSIDES drug info
ts_drug_annot = pd.read_csv('./data/twosides_drug_info.csv', index_col=0)
# TWOSIDES predicted expression
twosides_exp = pd.read_csv('./data/twosides_predicted_expression_scaled.csv')

In [ ]:
data_path = './data/'
train_x = pd.read_csv(data_path+'ddi_example_x.csv')
train_y = pd.read_csv(data_path+'ddi_example_y.csv')
print('Data loaded === ', train_x.shape, train_y.shape)

train_data = pd.concat([train_x, train_y], axis=1)

In [ ]:
def find_exp(drug_df, ts_exp, column_name):
    return pd.merge(drug_df, ts_exp, left_on=column_name, right_on='pubchem', how='left').iloc[:,2:]

def extract_expression(classification_model, train_data, drug_id, twosides_exp):

    att_a = train_data[(train_data.drug1 == drug_id)&(train_data.label == 1)].drop_duplicates(subset=['drug1','drug2'])
    att_b = train_data[(train_data.drug2 == drug_id)&(train_data.label == 1)].drop_duplicates(subset=['drug1','drug2'])
    
    # drug pair(a,b) & drug pair(b,a)
    att_b2 = att_b[['drug2','drug1','SE','label']]
    att_b2.columns = ['drug1','drug2','SE','label']
    
    att_drug_pair = pd.concat([att_a, att_b2], axis=0, ignore_index=True)
    
    # input
    drug1 = find_exp(att_drug_pair[['drug1']], twosides_exp, 'drug1')
    drug2 = find_exp(att_drug_pair[['drug2']], twosides_exp, 'drug2')
    drug_se = att_drug_pair['SE']
    print('Data shape: ', drug1.shape, drug2.shape, drug_se.shape)

    attention_layer = K.function([classification_model.layers[0].input, classification_model.layers[1].input, classification_model.layers[6].input],[classification_model.layers[7].output])
    attention_d1 = K.function([classification_model.layers[0].input, classification_model.layers[1].input, classification_model.layers[6].input],[classification_model.layers[4].output])
    attention_layer2 = K.function([classification_model.layers[0].input, classification_model.layers[1].input, classification_model.layers[6].input],[classification_model.layers[8].output])
    attention_d2 = K.function([classification_model.layers[0].input, classification_model.layers[1].input, classification_model.layers[6].input],[classification_model.layers[5].output])

    attention_vector = attention_layer([drug1, drug2, drug_se])[0]
    attention_vector2 = attention_layer2([drug1, drug2, drug_se])[0]
    d1_vector = attention_d1([drug1, drug2, drug_se])[0]
    d2_vector = attention_d2([drug1, drug2, drug_se])[0]

    se_list_gb_a = pd.DataFrame(pd.merge(att_a[['drug1','drug2']], train_data, on=['drug1','drug2'], how='left').groupby(['drug1','drug2'])['SE'].apply(list))
    se_list_gb_b = pd.DataFrame(pd.merge(att_b[['drug1','drug2']], train_data, on=['drug1','drug2'], how='left').groupby(['drug1','drug2'])['SE'].apply(list))
    
    se_list_gb = pd.concat([se_list_gb_a, se_list_gb_b], axis=0)
    
    print(se_list_gb.shape)
    se_binary = pd.DataFrame(np.zeros((se_list_gb.shape[0], 963)), columns=[x for x in range(0,963)], index=se_list_gb.index)
    for index, row in se_list_gb.iterrows():
        se_binary.loc[index,row.SE] = 1
    print('Corresponding side effects : ', se_binary.shape)    
    
    return att_drug_pair, attention_vector, attention_vector2, drug1, d1_vector, se_binary

In [ ]:
###################################
#    Extract weighted genes (Top 100)
#    convert index to corresponding gene IDs
###################################

def extract_top100genes(ddi_pair, gene_header):
    index_to_geneID = {k:v[0] for k,v in zip(gene_header.index, gene_header.values)}

    temp = pd.DataFrame(ddi_pair, columns=gene_header.values.flatten().tolist())
    top_100_gene = pd.DataFrame({col: temp.abs().T[col].nlargest(100).index.tolist() for n, col in enumerate(temp.T)}).T

    return top_100_gene

In [ ]:
drug_pair_subset, att_mat1, att_mat2, input_d1_mat, d1_att_mat, se_binary = extract_expression(classification_model, train_data, 3310, twosides_exp)

In [ ]:
drug1_sig_genes = extract_top100genes(att_mat1, gene_header)

In [ ]:
###################################
#    Heatmap of changed features
###################################

plt.figure()

temp = pd.DataFrame(d1_att_mat)
temp = temp.fillna(0)

# Clustermap
cg = sns.clustermap(temp, metric='correlation', figsize=(18,10), row_cluster=True, col_cluster=True, \
               cmap='seismic', vmin=-2, vmax=2, center=0, xticklabels=False, yticklabels=False)


hm_ax = cg.ax_heatmap
hm_ax.add_patch(Rectangle((0,0), 978,temp.shape[0], fill=False, edgecolor='grey', lw=3))
cg.ax_col_dendrogram.set_visible(False)

ax = cg.fig.add_axes([hm.x0 + hm.width+0.01, hm.y0, 0.01, hm.height])
den_row = cg.ax_row_dendrogram.get_position()
cg.ax_row_dendrogram.set_position([den_row.x0, den_row.y0, den_row.width*0.5, den_row.height])
cg.ax_heatmap.set_position([hm.x0 - den_row.width*0.5, hm.y0, hm.width+den_row.width*0.5, hm.height])

# Clustering
cluster_number = fcluster(cg.dendrogram_row.linkage, t=5, criterion='maxclust')[cg.dendrogram_row.reordered_ind]
cluster_number = cluster_number.reshape(cluster_number.shape[0], 1)

sns.heatmap(cluster_number,cmap=sns.color_palette('Accent_r', 10), cbar=False, xticklabels=False, yticklabels=False, ax=ax)

plt.show()

### Analysis 2. side effects of each cluster

In [ ]:
# The number of occurrence (drug pairs) of each side effect
se_occur = pd.read_csv('./data/twosides_side_effects_occurrence.csv')

# TWOSIDES side effect info
se_info = pd.read_csv('./data/twosides_side_effect_info.csv', index_col=0)
se_name_dict = dict(zip(se_info.SE_map, se_info['Side Effect Name']))

In [ ]:
# convert order based on clustering results from the heatmap
order = drug_pair_subset.reset_index(drop=True).loc[cg.dendrogram_row.reordered_ind][['drug1','drug2']]
se_ordered = pd.merge(order, se_binary.reset_index(), on=['drug1','drug2'])

In [ ]:
def nCk(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n - k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0
    
def hypergeometricTest(N, n, K, k):
    odds_ratio = (k/K) / (n/N)
    p_value = (nCk(K,k)*nCk(N-K, n-k)) / nCk(N,n)
    
    return p_value, odds_ratio

In [ ]:
# Count #side effects, #drugs for each cluster
num_pairs_per_cluster = pd.concat([se_ordered.iloc[:,2:], pd.DataFrame(cluster_number, columns=['ClusterNum'])], axis=1).groupby(['ClusterNum']).count()
num_se_per_cluster = pd.concat([se_ordered.iloc[:,2:], pd.DataFrame(cluster_number, columns=['ClusterNum'])], axis=1).groupby(['ClusterNum']).sum()


#############################################
#    Frequent side effects of each cluster
#############################################
se_freq_63460 = se_occur['num_occur'].to_dict()

se_freq_rank = pd.DataFrame(columns=[x for x in range(0,963)])
se_freq_rank_pvalue = pd.DataFrame(columns=[x for x in range(0,963)])

for index, row in num_pairs_per_cluster.iterrows():
    se_odds = dict()
    se_pvalue = dict()
    for key,value in dict(row).items():
        N = 63460
        n = int(se_freq_63460[key])
        K = value
        k = int(num_se_per_cluster.loc[index,key])

        p_value, odds_ratio = hypergeometricTest(N,n,K,k)
        
        se_odds[key] = odds_ratio
        se_pvalue[key] = p_value
        
    
    odds_df = pd.DataFrame(se_odds, index=[index])
    se_freq_rank = pd.concat([se_freq_rank, odds_df], axis=0)
    
    pvalue_df = pd.DataFrame(se_pvalue, index=[index])
    se_freq_rank_pvalue = pd.concat([se_freq_rank_pvalue, pvalue_df], axis=0)

In [ ]:
#######################################################################
#    TOP frequent side effects of each cluster (side effect names)
#######################################################################
topSE_cluster = pd.DataFrame({col: se_freq_rank.T[col].nlargest(5).index.tolist() for n, col in enumerate(se_freq_rank.T)}).T
topSE_cluster.replace(se_name_dict)